In [14]:
import findspark
findspark.init()

In [15]:
import pyspark

In [16]:
from pyspark.sql import SparkSession

In [17]:
spark = SparkSession.builder.appName("SocialNetworkAnalysis").config("spark.jars.packages", "C:/Big Data/graphframes/graphframes-0.8.2-spark3.0-s_2.12.jar").getOrCreate()
# spark = SparkSession.builder.appName("SocialNetworkAnalysis").config("spark.jars", "graphframes-0.8.2-spark3.0-s_2.12").getOrCreate()

In [18]:
spark

In [19]:
spark.sparkContext.setCheckpointDir("C:/Master-Projects/Social_Media_BigData/checkpoints")

In [20]:
df = spark.read.text("facebook_sample.txt.gz")

In [21]:
print(spark.sparkContext.getConf().get("spark.jars"))

C:/Big Data/graphframes/graphframes-0.8.3-spark3.4-s_2.12.jar


In [9]:
df2.show()

+-----+
|value|
+-----+
|  0 7|
| 0 22|
| 0 27|
| 0 37|
| 0 54|
| 0 57|
| 0 59|
| 0 70|
| 0 72|
| 0 78|
| 0 80|
| 0 86|
| 0 92|
| 0 97|
| 0 99|
|0 102|
|0 104|
|0 105|
|0 110|
|0 112|
+-----+
only showing top 20 rows



In [22]:
from pyspark.sql.functions import split, col
from graphframes import GraphFrame

In [23]:
edges = df2.withColumn("src", split(col("value"), " ")[0]).withColumn("dst", split(col("value"), " ")[1]).select("src", "dst")

In [24]:
vertices = edges.select(col("src").alias("id")).union(edges.select(col("dst").alias("id"))).distinct()

In [25]:
graph = GraphFrame(vertices, edges)

Py4JJavaError: An error occurred while calling o200.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.jav

* a) Si mund të identifikojmë përdoruesit më të ndikshëm duke përdorur algoritme të centralitetit?

In [14]:
# Apliko PageRank
pagerank = graph.pageRank(resetProbability=0.15, maxIter=10)
pagerank.vertices.orderBy(col("pagerank").desc()).show(10, truncate=False)

C:\spark\spark-3.5.4-bin-hadoop3\spark-3.5.4-bin-hadoop3\python\pyspark\sql\dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+----+------------------+
|id  |pagerank          |
+----+------------------+
|1902|28.019976979035754|
|2655|19.642247577430364|
|2598|15.243826879741622|
|1891|12.069809736701393|
|1898|12.028756603786434|
|2649|11.68518961150843 |
|1888|11.212094077108803|
|1864|11.073874826459907|
|2654|10.499073941303413|
|2646|10.14754570934359 |
+----+------------------+
only showing top 10 rows



b) Si mund të përdoren grafët për të zbuluar grupe të ngjashme përdoruesish në një rrjet social?

In [52]:
graph.edges.show()

+---+---+
|src|dst|
+---+---+
|  0|  7|
|  0| 22|
|  0| 27|
|  0| 37|
|  0| 54|
|  0| 57|
|  0| 59|
|  0| 70|
|  0| 72|
|  0| 78|
|  0| 80|
|  0| 86|
|  0| 92|
|  0| 97|
|  0| 99|
|  0|102|
|  0|104|
|  0|105|
|  0|110|
|  0|112|
+---+---+
only showing top 20 rows



In [53]:
graph.vertices.show()

+----+
|  id|
+----+
| 829|
|1090|
|1159|
|1436|
|1512|
|1572|
|2069|
|2088|
|2136|
|2162|
|2294|
|2904|
|3210|
|3606|
| 125|
| 451|
| 800|
| 944|
|1669|
|2110|
+----+
only showing top 20 rows



In [17]:
spark.sparkContext.getConf().get("spark.jars")

'C:/Big Data/graphframes/graphframes-0.8.2-spark3.0-s_2.12.jar'

In [15]:
from pyspark.storagelevel import StorageLevel

graph.persist(StorageLevel.MEMORY_AND_DISK)  # Store graph in memory & disk if needed

connected_components = graph.connectedComponents()

Py4JJavaError: An error occurred while calling o102.run.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputCommitt

In [ ]:
connected_components.show()

In [15]:
# Gjej komponentët e lidhur
# spark.sparkContext.setCheckpointDir(r"C:\Big Data\checkpoint")
graph = GraphFrame(vertices, edges)
connected_components = graph.connectedComponents()
connected_components.orderBy("component").show(10, truncate=False)

AnalysisException: [PATH_ALREADY_EXISTS] Path http://laptop-0sn8dct8:4040/fitor/spark/checkpoints/21b124f7-017b-4545-ab88-35af9936d7b4/connected-components-2466ec80/2 already exists. Set mode as "overwrite" to overwrite the existing path.

c) Si mund të përdorim algoritmet e përhapjes së informacionit për të analizuar shpërndarjen e lajmeve ose viruseve?

In [ ]:
# Apliko algoritmin e përhapjes së informacionit
label_propagation = graph.labelPropagation(maxIter=5)
label_propagation.orderBy("label").show(10, truncate=False)

d) Çfarë ndikimi ka heqja e një përdoruesi kyç në strukturën dhe stabilitetin e rrjetit social?

In [ ]:
# Hiq një përdorues nga grafi
user_to_remove = "0"  # P.sh., përdoruesi me ID 0
updated_edges = edges.filter((col("src") != user_to_remove) & (col("dst") != user_to_remove))
updated_vertices = vertices.filter(col("id") != user_to_remove)
updated_graph = GraphFrame(updated_vertices, updated_edges)

# Analizo strukturën e re
updated_graph.connectedComponents().show()

e) Si mund të përdorim PageRank për të vlerësuar ndikimin e përdoruesve në një rrjet social?

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from graphframes import GraphFrame
import json

# Inicimi i Spark
sc = SparkContext("local", "Social Network Analysis")
spark = SparkSession(sc)

# Leximi i skedarit edges.csv
edges = spark.read.csv("chameleon/musae_chameleon_edges.csv", header=True, inferSchema=True)
edges = edges.withColumnRenamed("id1", "src").withColumnRenamed("id2", "dst")

# Leximi i skedarit target.csv
targets = spark.read.csv("chameleon/musae_chameleon_target.csv", header=True, inferSchema=True)

# Leximi i skedarit features.json
with open("chameleon/musae_chameleon_features.json", "r") as f:
    features_data = json.load(f)

# Kthimi i JSON në DataFrame
features = spark.createDataFrame([(k, v) for k, v in features_data.items()], ["id", "features"])

# Konvertimi i kolonës 'features' në listë
from pyspark.sql.functions import col
features = features.withColumn("features", col("features").cast("array<int>"))

# Printimi për kontroll
edges.show()
targets.show()
features.show()


+----+----+
| src| dst|
+----+----+
|2034|1939|
|2034|2263|
|2034|1976|
|2034|2234|
|2034| 804|
|2034|1474|
|2034|2110|
|2034| 652|
|2034|1350|
|2034|1860|
|2034|2243|
|2034|1889|
|2034|1356|
|2034| 536|
|1747|1554|
|1747|1843|
|1747|1612|
|1747|1776|
|1747|1911|
| 817| 115|
+----+----+
only showing top 20 rows

+---+------+
| id|target|
+---+------+
|  0|   171|
|  1|  8089|
|  2|  8568|
|  3| 47178|
|  4|  3634|
|  5|   215|
|  6|  1054|
|  7|  1106|
|  8|   109|
|  9|  3863|
| 10|   590|
| 11|   175|
| 12|  1399|
| 13|    39|
| 14|   444|
| 15| 64492|
| 16| 62284|
| 17|154527|
| 18|   673|
| 19|    33|
+---+------+
only showing top 20 rows



Py4JJavaError: An error occurred while calling o60.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6) (LAPTOP-0SN8DCT8 executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=2, The system cannot find the file specified
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:181)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.r

In [9]:
with open("C:/Master-Projects/Social_Media_BigData/chameleon/musae_chameleon_features.json", "r") as f:
    features_data = json.load(f)

In [16]:
features_data

{'2098': [1765,
  3047,
  630,
  1308,
  2455,
  2623,
  1883,
  1810,
  934,
  2111,
  288,
  2077,
  642,
  2405,
  3025,
  437,
  86,
  649,
  860,
  2784,
  1231,
  430,
  830,
  216,
  1051,
  2980,
  1787,
  1324],
 '47': [3001,
  2690,
  2299,
  1312,
  1109,
  721,
  1721,
  2887,
  1794,
  2297,
  2753,
  2539,
  415,
  598,
  302,
  1846,
  2090,
  1297,
  1488,
  1674,
  1316,
  1834],
 '1674': [2114, 2518, 1794, 768, 2479, 997, 849],
 '1207': [1855,
  1073,
  251,
  1000,
  1880,
  953,
  1794,
  2297,
  2753,
  2539,
  598,
  528,
  1319,
  847,
  2899],
 '1340': [2357,
  2453,
  2468,
  73,
  584,
  1001,
  165,
  372,
  988,
  178,
  666,
  2201,
  1300,
  2441,
  370,
  971,
  2158,
  1130,
  1884,
  2645,
  2038,
  2364],
 '196': [1079,
  2802,
  2468,
  73,
  584,
  165,
  2826,
  599,
  655,
  2070,
  923,
  191,
  207,
  1677,
  2645,
  2948,
  2116,
  1669,
  2716],
 '1586': [1157,
  245,
  2620,
  1347,
  73,
  584,
  372,
  1008,
  2492,
  1161,
  1026,
  1733,
 

In [11]:
from pyspark.sql import Row

In [12]:
data = [{"id": int(k), "features": v} for k, v in features_data.items()]

In [17]:
data

[{'id': 2098,
  'features': [1765,
   3047,
   630,
   1308,
   2455,
   2623,
   1883,
   1810,
   934,
   2111,
   288,
   2077,
   642,
   2405,
   3025,
   437,
   86,
   649,
   860,
   2784,
   1231,
   430,
   830,
   216,
   1051,
   2980,
   1787,
   1324]},
 {'id': 47,
  'features': [3001,
   2690,
   2299,
   1312,
   1109,
   721,
   1721,
   2887,
   1794,
   2297,
   2753,
   2539,
   415,
   598,
   302,
   1846,
   2090,
   1297,
   1488,
   1674,
   1316,
   1834]},
 {'id': 1674, 'features': [2114, 2518, 1794, 768, 2479, 997, 849]},
 {'id': 1207,
  'features': [1855,
   1073,
   251,
   1000,
   1880,
   953,
   1794,
   2297,
   2753,
   2539,
   598,
   528,
   1319,
   847,
   2899]},
 {'id': 1340,
  'features': [2357,
   2453,
   2468,
   73,
   584,
   1001,
   165,
   372,
   988,
   178,
   666,
   2201,
   1300,
   2441,
   370,
   971,
   2158,
   1130,
   1884,
   2645,
   2038,
   2364]},
 {'id': 196,
  'features': [1079,
   2802,
   2468,
   73,
   584,
   

In [18]:
# Transformimi i të dhënave JSON në DataFrame
features_df = spark.createDataFrame(Row(**x) for x in data)

In [23]:
from pyspark.sql.types import IntegerType, ArrayType

features_df = features_df.withColumn("id", features_df["id"].cast(IntegerType()))
features_df = features_df.withColumn("features", features_df["features"].cast(ArrayType(IntegerType())))

features_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [25]:
# Krijimi i DataFrame për vertices duke kombinuar target dhe features
vertices = targets.join(features, on="id", how="inner")

# Krijimi i GraphFrame
graph = GraphFrame(vertices, edges)

# Kontrollimi i strukturës së grafit
graph.vertices.show()
graph.edges.show()

C:\Python310\lib\site-packages\pyspark\sql\dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling o268.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.jav

In [26]:
# Krijimi i DataFrame për vertices duke kombinuar target dhe features
vertices = targets.join(features_df, on="id", how="inner")

# Krijimi i GraphFrame
graph = GraphFrame(vertices, edges)

# Kontrollimi i strukturës së grafit
graph.vertices.show()
graph.edges.show()

Py4JJavaError: An error occurred while calling o274.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.jav

In [15]:
import gzip

file_path = "facebook_combined.txt.gz"

with gzip.open(file_path, 'rt') as f:
    num_lines = sum(1 for _ in f)

print(f"Total edges: {num_lines}")


Total edges: 88234


In [1]:
import gzip
import random

file_path = "facebook_sample4.txt.gz"
sample_path = "facebook_sample5.txt.gz"
sample_ratio = 0.1  # Keep 10% of the data

with gzip.open(file_path, 'rt') as f_in, gzip.open(sample_path, 'wt') as f_out:
    for line in f_in:
        if random.random() < sample_ratio:
            f_out.write(line)

print("Sample file created.")


Sample file created.
